In [16]:
from Parse import *

In [32]:
notes = [
    ' A_LINE No trains between 168 St and 207 St',
    ' 2_LINE  5_LINE Uptown trains run express from 3 Av-149 St to E 180 St',
    ' 2_LINE Downtown trains run local from 96 St to Times Sq-42 St',
    ' F_LINE Queens-bound trains skip Roosevelt Island and Lexington Av',
    ' 2_LINE trains replace weekend  5_LINE service between Dyre Av and E 180 St',
    ' 4_LINE All trains from Junius St to Sutter Av-Rutland Rd board at the New Lots Av-bound platform',
    ' 4_LINE No trains between Utica Av and New Lots Av',
    ' A_LINE Trains make local stops in both directions at 155 St and 163 St',
    ' D_LINE  N_LINE  R_LINE Prospect Av is closed for renovation',
    ' F_LINE Jamaica-bound trains are rerouted via the  E_LINE after 47-50 Sts to Roosevelt Av',
    ' F_LINE Manhattan-bound platforms at Avenue U, Avenue P, Avenue N, Bay Pkwy  and Avenue I are closed for renovation',
    ' L_LINE Service operates every 12 minutes between Myrtle-Wyckoff Avs and Rockaway Pkwy',
    ' M_LINE Service ends early between 71 Av and Essex St E_LINE  F_LINE  R_LINE trains provide alternate service',
    ' R_LINE Service restored at Bay Ridge Av',
    ' sir_LINE Grant City Station Entrance Closure'
    ]

In [33]:
data = []
for i in notes:
    data.append(parse(i))

In [34]:
t = data[0]
test1 = data[0]
test2 = data[3]
test3 = data[8]
test3

{'lines': ['D_LINE', 'N_LINE', 'R_LINE'],
 'reason': 'renovation',
 'stations': [''],
 'type': 'close'}

In [20]:
import numpy as np
d = np.load('stations_dic.npy').item()
d['A_LINE'].head()

NAME LINE       lang       long top_to_bottom reason1  \
0  Far Rockaway-Mott Av  [A]  40.603995 -73.755405           0.0     0.0   
1           Beach 25 St  [A]  40.600066 -73.761353           0.0     0.0   
2           Beach 36 St  [A]  40.595398 -73.768175           0.0     0.0   
3           Beach 44 St  [A]  40.592943 -73.776013           0.0     0.0   
4           Beach 60 St  [A]  40.592374 -73.788522           0.0     0.0   

  bottom_to_top reason2                 name_split  
0           0.0     0.0  [Far, Rockaway, Mott, Av]  
1           0.0     0.0            [Beach, 25, St]  
2           0.0     0.0            [Beach, 36, St]  
3           0.0     0.0            [Beach, 44, St]  
4           0.0     0.0            [Beach, 60, St]

##### update function

In [21]:
def compare(ls, pool):
    target = list(map(str, ls))
    return np.sum([a in pool for a in target])

In [22]:
def find_stops(dataframe, words):
    
    df = dataframe.copy()
    
    splt = words.replace('-', ' ').split()
    
    result = df.name_split.apply(lambda x: compare(splt, x))
    
    index = np.argmax(result)

    if np.sum([result == max(result)]) == 1:
        return index
    else:
        print("Can't find the station")
        return

In [23]:
list(range(11,5,-1))

[11, 10, 9, 8, 7, 6]

In [24]:
t

{'from': '168 St', 'lines': ['A_LINE'], 'to': '207 St', 'type': 'No train'}

In [25]:
top_to_bottom = ['Flatbush Av-bound','Manhattan-bound']
bottom_to_top = ['Queens-bound']

In [26]:
def update(note):
    
    tp = note['type']
    
    ## SKIP
    if tp == 'skip':
        
        line_names = note['lines']
        stations = note['stations']
        bound = note['direction']
        
        ## update for each line
        for line in line_names:
            df = d[line] 
            
            ## find stations
            for s in stations:
                
                ## try to find stations directly
                ind = df[df.NAME == s].index
                
                if len(ind) == 0:  ## if not match, find the closest one. 
                    ind = find_stops(df, s)
                
                if ind <= len(df):
                    
                    if bound == []:
                        d[line].set_value(ind, 'top_to_bottom', 'Not working')  ## update status
                        d[line].set_value(ind, 'reason1', 'skip')  ## update reason

                        d[line].set_value(ind, 'bottom_to_top', 'Not working')  ## update status
                        d[line].set_value(ind, 'reason2', 'skip')  ## update reason
                        
                    
                    
                    elif bound in top_to_bottom:  ##check direction
                        
                        d[line].set_value(ind, 'top_to_bottom', 'Not working')  ## update status
                        d[line].set_value(ind, 'reason1', 'skip')  ## update reason

                    elif bound in bottom_to_top: 
                        d[line].set_value(ind, 'bottom_to_top', 'Not working')  ## update status
                        d[line].set_value(ind, 'reason2', 'skip')  ## update reason
        
        
        
        
    elif tp == 'No train':
        
        line_names = note['lines']
        from_ = note['from']
        to_ = note['to']
            
        ## update for each line
        for line in line_names:
            df = d[line] 
            
            if from_ == 'all stations':     ## check if all stations have no train running
                
                ind = d[line].index
                
                d[line].set_value(ind, 'top_to_bottom', 'Not working')  ## update status
                d[line].set_value(ind, 'reason1', 'no train')  ## update reason

                d[line].set_value(ind, 'bottom_to_top', 'Not working')  ## update status
                d[line].set_value(ind, 'reason2', 'no train')  ## update reason
            
            else:
                ## find start station

                from_id = df[df.NAME == from_].index
                if len(from_id) == 0:
                    from_id = find_stops(df, from_)

                ## find end station
                end_id = df[df.NAME == to_].index
                if len(end_id) == 0:
                    end_id = find_stops(df, to_)
                
                ## all stops
                if from_id[0] <= end_id:
                    ind = list(range(from_id[0], end_id))
                else:
                    ind = list(range(end_id, from_id[0]))

                d[line].set_value(ind, 'top_to_bottom', 'Not working')  ## update status
                d[line].set_value(ind, 'reason1', 'no train')  ## update reason

                d[line].set_value(ind, 'bottom_to_top', 'Not working')  ## update status
                d[line].set_value(ind, 'reason2', 'no train')  ## update reason
            
    return 

In [27]:
update(test1)
update(test2)

In [29]:
d['F_LINE']

NAME          LINE       lang       long  \
0                 Jamaica-179 St           [F]  40.712646 -73.783817   
1                         169 St           [F]  40.710518 -73.793474   
2                   Parsons Blvd           [F]  40.707572 -73.803289   
3                   Sutphin Blvd           [F]  40.705418 -73.810833   
4        Briarwood-Van Wyck Blvd        [E, F]  40.709162 -73.820693   
5         Kew Gardens-Union Tpke        [E, F]  40.714035 -73.830370   
6                          75 Av        [E, F]  40.718045 -73.836924   
7             Forest Hills-71 Av  [E, F, M, R]  40.721594 -73.844517   
8       Jackson Hts-Roosevelt Av  [E, F, M, R]  40.746540 -73.891299   
9             21 St-Queensbridge           [F]  40.753739 -73.941938   
10      Roosevelt Island-Main St           [F]  40.759172 -73.953235   
11            Lexington Av-63 St        [F, Q]  40.764618 -73.966090   
12                         57 St           [F]  40.764085 -73.977368   
13     47-50 Sts-Rockefeller Ctr  [B, D, F, M]  40.758641 -73.981331   
14               42 St-Bryant Pk  [B, D, F, M]  40.754184 -73.984591   
15               Herald Sq-34 St  [B, D, F, M]  40.749789 -73.987772   
16                         23 St        [F, M]  40.742954 -73.992765   
17                         14 St        [F, M]  40.738228 -73.996209   
18  W 4 St-Washington Sq (Lower)  [B, D, F, M]  40.732254 -74.000308   
19         Broadway-Lafayette St  [B, D, F, M]  40.725297 -73.996204   
20          Lower East Side-2 Av           [F]  40.723402 -73.989938   
21          Delancey St-Essex St           [F]  40.718681 -73.988078   
22                 East Broadway           [F]  40.713855 -73.990177   
23                       York St           [F]  40.699743 -73.986885   
24              Jay St-MetroTech     [A, C, F]  40.692471 -73.987218   
25                     Bergen St        [F, G]  40.686111 -73.990756   
26                    Carroll St        [F, G]  40.680273 -73.994947   
27                   Smith-9 Sts        [F, G]  40.673641 -73.995892   
28                     4 Av-9 St        [F, G]  40.670272 -73.989779   
29                          7 Av        [F, G]  40.666245 -73.980251   
30           15 St-Prospect Park        [F, G]  40.660036 -73.979736   
31              Ft Hamilton Pkwy        [F, G]  40.650782 -73.975776   
32                     Church Av        [F, G]  40.644272 -73.979721   
33                     Ditmas Av           [F]  40.636119 -73.978172   
34                         18 Av           [F]  40.629755 -73.976971   
35                      Avenue I           [F]  40.625017 -73.976069   
36                       Bay Pky           [F]  40.620732 -73.975257   
37                      Avenue N           [F]  40.614357 -73.974049   
38                      Avenue P           [F]  40.608843 -73.973003   
39                     Kings Hwy           [F]  40.603258 -73.972355   
40                      Avenue U           [F]  40.595925 -73.973376   
41                      Avenue X           [F]  40.589450 -73.974266   
42                    Neptune Av           [F]  40.580739 -73.974593   
43            W 8 St-NY Aquarium        [F, Q]  40.576034 -73.975958   
44     Coney Island-Stillwell Av  [D, F, N, Q]  40.577281 -73.981236   

   top_to_bottom reason1 bottom_to_top reason2  \
0            0.0     0.0           0.0     0.0   
1            0.0     0.0           0.0     0.0   
2            0.0     0.0           0.0     0.0   
3            0.0     0.0           0.0     0.0   
4            0.0     0.0           0.0     0.0   
5            0.0     0.0           0.0     0.0   
6            0.0     0.0           0.0     0.0   
7            0.0     0.0           0.0     0.0   
8            0.0     0.0           0.0     0.0   
9            0.0     0.0           0.0     0.0   
10           0.0     0.0   Not working    skip   
11           0.0     0.0   Not working    skip   
12           0.0     0.0           0.0     0.0   
13           0.

In [30]:
d['A_LINE']

NAME          LINE       lang       long  \
0            Far Rockaway-Mott Av           [A]  40.603995 -73.755405   
1                     Beach 25 St           [A]  40.600066 -73.761353   
2                     Beach 36 St           [A]  40.595398 -73.768175   
3                     Beach 44 St           [A]  40.592943 -73.776013   
4                     Beach 60 St           [A]  40.592374 -73.788522   
5                     Beach 67 St           [A]  40.590927 -73.796924   
6                     Beach 90 St        [A, S]  40.588092 -73.813651   
7                     Beach 98 St        [A, S]  40.585386 -73.820521   
8                    Beach 105 St        [A, S]  40.583268 -73.827581   
9      Rockaway Park-Beach 116 St        [A, S]  40.580956 -73.835590   
10                  Broad Channel        [A, S]  40.608402 -73.815833   
11       Howard Beach-JFK Airport           [A]  40.660476 -73.830301   
12      Aqueduct-North Conduit Av           [A]  40.668234 -73.834058   
13             Aqueduct Racetrack           [A]  40.672097 -73.835919   
14                         104 St        [A, S]  40.681711 -73.837683   
15                         111 St        [A, S]  40.684331 -73.832163   
16       Ozone Park-Lefferts Blvd        [A, S]  40.685951 -73.825798   
17                  Rockaway Blvd        [A, S]  40.680429 -73.843853   
18                          88 St        [A, S]  40.679843 -73.851470   
19                          80 St        [A, S]  40.679371 -73.858992   
20                       Grant Av        [A, S]  40.677044 -73.865050   
21                      Euclid Av     [A, C, S]  40.675377 -73.872106   
22                    Shepherd Av        [A, C]  40.674130 -73.880750   
23                  Van Siclen Av        [A, C]  40.672710 -73.890358   
24                     Liberty Av        [A, C]  40.674542 -73.896548   
25              Broadway Junction        [A, C]  40.678334 -73.905316   
26                    Rockaway Av        [A, C]  40.678340 -73.911946   
27                       Ralph Av        [A, C]  40.678822 -73.920786   
28                       Utica Av        [A, C]  40.679364 -73.930729   
29            Kingston-Throop Avs        [A, C]  40.679919 -73.940859   
..                            ...           ...        ...        ...   
36                        High St        [A, C]  40.699337 -73.990531   
37                      Fulton St        [A, C]  40.710162 -74.007623   
38                    Chambers St        [A, C]  40.714111 -74.008585   
39        Canal St-Holland Tunnel     [A, C, E]  40.720824 -74.005229   
40                      Spring St     [A, C, E]  40.726227 -74.003739   
41   W 4 St-Washington Sq (Upper)     [A, C, E]  40.732338 -74.000495   
42                          14 St     [A, C, E]  40.740893 -74.001690   
43                          23 St     [A, C, E]  40.745906 -73.998041   
44             34 St-Penn Station     [A, C, E]  40.752287 -73.993391   
45  42 St-Port Authority Bus Term     [A, C, E]  40.757308 -73.989735   
46                          50 St     [A, C, E]  40.762456 -73.985984   
47          59 St-Columbus Circle  [A, B, C, D]  40.768250 -73.981649   
48                          72 St     [A, B, C]  40.775519 -73.976337   
49                          81 St     [A, B, C]  40.781346 -73.972098   
50                          86 St     [A, B, C]  40.785823 -73.968828   
51                          96 St     [A, B, C]  40.791619 -73.964602   
52                         103 St     [A, B, C]  40.796061 -73.961370   
53        Cathedral Pkwy (110 St)     [A, B, C]  40.800582 -73.958067   
54                         116 St     [A, B, C]  40.805058 -73.954798   
55                         125 St  [A, B, C, D]  40.811072 -73.952248   
56                         135 St     [A, B, C]  40.817906 -73.947535   
57                         145 St  [A, B, C, D]  40.824766 -73.944088   
58                         155 St        [A, C]  40.830518 -73.941514   
59            